In [2]:
import torch
import torch.nn as nn
import numpy as np
import cv2
from torchvision import transforms
from PIL import Image
import os
from torchvision import models

In [3]:
# Define a custom ResNet model with added dropout and a fully connected layer
class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.model = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V2)  # Load pre-trained ResNet101 model
        self.model.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(self.model.fc.in_features, num_classes),
        )

    def forward(self, x):
        return self.model(x)

In [4]:
# Function to load the model weights
def load_model(model_path, num_classes):
    model = CustomResNet(num_classes)
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()  # Set the model to evaluation mode
    return model

In [11]:
# Function to preprocess the image
def preprocess_image(img_path, target_size):
    img = Image.open(img_path).convert('RGB')  # Convert to RGB
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension
    return img_tensor

# Function to make predictions
def predict_image(model, img_tensor, class_names):
    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted_class = torch.max(outputs, 1)
        confidence = torch.nn.functional.softmax(outputs, dim=1)[0][predicted_class]
    return class_names[predicted_class.item()], confidence.item()

# Load your trained model
model_path ="C:\\Users\\Acer\\Desktop\\Model\\best_model1.pth"
model = load_model(model_path, num_classes=15)

# Define class names (replace with your actual class names)
class_names = ['Pneumonia','Atelectasis','Cardiomegaly', 'Consolidation','Edema','Effusion', 'Emphysema','Fibrosis','Infiltration','Mass','Nodule','Pleural_Thickening','Pneumothorax', 'Normal', 'Hernia']

# Path to the single image
img_path = "C:\\Users\\Acer\\Desktop\\Model\\consolidation.jpg"  # Replace with your image path

# Preprocess the image
img_tensor = preprocess_image(img_path, target_size=(224, 224))  # Adjust size as needed

# Make a prediction
predicted_class, confidence = predict_image(model, img_tensor, class_names)

# Display the result
print(f'Image: {img_path}')
print(f'Predicted Class: {predicted_class}')
print(f'Confidence: {confidence * 100:.2f}%')

Image: C:\Users\Acer\Desktop\Model\consolidation.jpg
Predicted Class: Infiltration
Confidence: 89.06%
